In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Conv2D
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pickle

img_height =255
img_width =255
batch_size = 64
AUTOTUNE = tf.data.AUTOTUNE
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
    )
num_classes = metadata.features['label'].num_classes
label_name = metadata.features['label'].names
print(label_name, ", classnum : ", num_classes)

['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses'] , classnum :  5


In [14]:
def prepare(ds, shuffle=False, augment=False):
    preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input
    ds = ds.map(lambda x,y : (tf.image.resize(x,[img_height, img_width]), y),
                num_parallel_calls=AUTOTUNE)
    
    #전처리 적용
    ds = ds.map(lambda x,y : (preprocess_input(x), y),
            num_parallel_calls=AUTOTUNE)
    
    #batch all datasets
    ds = ds.batch(batch_size)
    
    if augment:
        data_agumentation = tf.keras.Sequential([
            layers.RandomFlip("horizontal_and_vertical"),
            layers.RandomRotation(0.2),
        ])
        ds = ds.map(lambda x,y : (data_agumentation(x, training= True), y),
                    num_parallel_calls=AUTOTUNE)
        
    return ds.prefetch(buffer_size =AUTOTUNE)

In [15]:
train_ds = prepare(train_ds, shuffle=True , augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

In [16]:
base_model = tf.keras.applications.MobileNetV3Small(
    weights = 'imagenet',
    input_shape = (img_height, img_width, 3),
    include_top = False)
base_model.trainable = False
inputs = tf.keras.Input(shape=(img_height, img_width,3))

x = base_model(inputs, training =False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(num_classes,
                                activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

/home/so/intro/.venv/lib/python3.10/site-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


In [17]:
model.summary()
model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
history = model.fit(train_ds, epochs=30, validation_data=val_ds)
model.save('transfer_learning_flower.keras')
with open('history_flower', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 255, 255, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobileNetV3Small (Functional)   │ (None, 8, 8, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │         2,885 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 942,005 (3.59 MB)

 Trainable params: 2,885 (11.27 KB)

 Non-trainable params: 939,120 (3.58 MB)

Epoch 1/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 9s 153ms/step - accuracy: 0.3490 - loss: 1.6016 - val_accuracy: 0.7520 - val_loss: 0.7568
Epoch 2/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 135ms/step - accuracy: 0.7122 - loss: 0.8126 - val_accuracy: 0.8365 - val_loss: 0.5427
Epoch 3/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 134ms/step - accuracy: 0.7956 - loss: 0.6064 - val_accuracy: 0.8610 - val_loss: 0.4698
Epoch 4/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 134ms/step - accuracy: 0.7968 - loss: 0.5560 - val_accuracy: 0.8719 - val_loss: 0.4164
Epoch 5/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 7s 139ms/step - accuracy: 0.8306 - loss: 0.4988 - val_accuracy: 0.8774 - val_loss: 0.3855
Epoch 6/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 7s 139ms/step - accuracy: 0.8339 - loss: 0.4702 - val_accuracy: 0.8801 - val_loss: 0.3669
Epoch 7/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 135ms/step - accuracy: 0.8527 - loss: 0.4401 - val_accuracy: 0.8883 - val_loss: 0.3503
Epoch 8/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 7s 141ms/step - accuracy: 0.8614 - loss: 0.4017 - val_accuracy: 0.